### BQ Load 

In [7]:
dataset_id = "aero_staging"

In [8]:
!bq --location=US mk --dataset {dataset_id}

BigQuery error in mk operation: Dataset 'probable-pager-266720:aero_staging'
already exists.


In [9]:
!bq --location=US load --autodetect --skip_leading_rows=1 \
--source_format=CSV {dataset_id}.birds_eye \
'gs://areo-data/AERO-BirdsEye-Data.csv' 

Waiting on bqjob_r1268eaf45a535339_0000017194bda074_1 ... (2s) Current status: DONE   


### Modeled Table

In [6]:
%%bigquery 

create table aero_modeled.birds_eye as
select nct_number, sponsor, title, start_year, start_month, phase, condition
from aero_staging.birds_eye

""


#### Primary Keys

In [12]:
%%bigquery

select count(*) as count from aero_modeled.birds_eye 
union all 
select count(distinct nct_number) as counter from aero_modeled.birds_eye 

,count
0,27496
1,13748


#### Duplicates

In [13]:
%%bigquery

select nct_number, count(nct_number) as count
from aero_modeled.birds_eye 
group by nct_number
having count(nct_number) > 1 limit 10;

,nct_number,count
0,NCT00142558,2
1,NCT00798369,2
2,NCT00819585,2
3,NCT00927810,2
4,NCT01029652,2
5,NCT01080131,2
6,NCT02187029,2
7,NCT00141401,2
8,NCT00150423,2
9,NCT00156078,2


### Beam

In [14]:
%run birds_eye_beam.py 

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []
INFO:apache_beam.runners.direct.direct_runner:Running pipeline with DirectRunner.
INFO:oauth2client.transport:Refreshing due to a 401 (attempt 1/2)
INFO:apache_beam.io.gcp.bigquery_tools:Using location 'US' from table <TableReference
 datasetId: 'aero_modeled'
 projectId: 'probable-pager-266720'
 tableId: 'birds_eye'> referenced by query SELECT nct_number, sponsor, title, start_year, start_month, phase, condition FROM aero_modeled.birds_eye limit 10
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with num_shards: 1 (skipped: 0), batches: 1, num_threads: 1
INFO:apache_beam.io.filebasedsink:Renamed 1 shards in 0.10 seconds.
INFO:apache_beam.io.filebasedsink:Starting finalize_write threads with nu

In [ ]:
%run birds_eye_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))
INFO:apache_beam.runners.dataflow.internal.apiclient:Starting GCS upload to gs://ncorona-lyme-ncl/staging/birds-eye-df.1587864892.810286/pipeline.pb...
INFO:apache_beam.runners.dataflow.internal.apiclient:Completed GCS upload to gs://ncorona-lyme-ncl/staging/birds-eye-df.1587864892.810286/pipeline.pb in 0 seconds.
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/home/jupyter/venv/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmpf8hbx7k7', 'apache-beam==2.19.0', '--no-deps', '--no-binary', ':all:']
INFO:apache_beam.runners.portability.stager:Staging SDK sources from PyPI to gs://ncorona-lyme-ncl/staging/birds-eye-df.1587864892.810286/dataflow_pyth

#### Primary Key & Duplicate Checks

In [4]:
%%bigquery

select count(*) as count from aero_modeled.birds_eye_Beam_DF
union all 
select count(distinct nct_number) as count from aero_modeled.birds_eye_Beam_DF

,count
0,13748
1,13748


In [5]:
%%bigquery

select nct_number, count(nct_number) as count
from aero_modeled.birds_eye_Beam_DF
group by nct_number
having count(nct_number) > 1 limit 10;

,nct_number,count
